# <a id='toc1_'></a>[**Data Quality Visualization**](#toc0_)

This notebook is for visualizing data quality and checking how acceleration data is associated with signal quality as well as the performance of beat correction algorithms.

Required data can be installed from [here](https://www.dropbox.com/scl/fo/89iubvi2wediqwkla8zqe/AMQt1_65qKiwMd4saxfz66M?rlkey=7r8ck6rnm0d0hekhsq1zdmrvm&st=1g6qz2du&dl=0).

**Table of contents**<a id='toc0_'></a>    
- [**Data Quality Visualization**](#toc1_)    
  - [**Import libraries**](#toc1_1_)    
  - [**Setting parameters**](#toc1_2_)    
  - [**Visualize raw data**](#toc1_3_)    
    - [Clean ECG](#toc1_3_1_)    
    - [Noisy PPG](#toc1_3_2_)    
    - [Noisy PPG 2](#toc1_3_3_)    
    - [Noisy ECG](#toc1_3_4_)    
    - [Noisy ECG 2](#toc1_3_5_)    
    - [Unusable ECG](#toc1_3_6_)    
  - [**Visualize artifacts**](#toc1_4_)    
    - [**Artifact detection methods**](#toc1_4_1_)    
    - [Clean ECG](#toc1_4_2_)    
    - [Noisy PPG](#toc1_4_3_)    
    - [Noisy PPG 2](#toc1_4_4_)    
    - [Noisy ECG](#toc1_4_5_)    
    - [Noisy ECG 2](#toc1_4_6_)    
    - [Unusable ECG](#toc1_4_7_)    
  - [**Beat correction**](#toc1_5_)    
    - [Clean ECG](#toc1_5_1_)    
    - [Noisy PPG](#toc1_5_2_)    
    - [Noisy PPG 2](#toc1_5_3_)    
    - [Noisy ECG](#toc1_5_4_)    
    - [Noisy ECG 2](#toc1_5_5_)    
    - [Unusable ECG](#toc1_5_6_)    
  - [**Visualize the association between acceleration data and signal quality**](#toc1_6_)    
    - [Noisy ECG](#toc1_6_1_)    
      - [Segment size = 300 seconds](#toc1_6_1_1_)    
      - [Segment size = 60 seconds](#toc1_6_1_2_)    
    - [Noisy ECG 2](#toc1_6_2_)    
      - [Segment size = 300 seconds](#toc1_6_2_1_)    
      - [Segment size = 60 seconds](#toc1_6_2_2_)    
    - [Unusable ECG](#toc1_6_3_)    
      - [Segment size = 300 seconds](#toc1_6_3_1_)    
      - [Segment size = 60 seconds](#toc1_6_3_2_)    
  - [**Visualize the association between acceleration data and beat correction**](#toc1_7_)    
    - [Noisy ECG](#toc1_7_1_)    
      - [Segment size = 300 seconds](#toc1_7_1_1_)    
      - [Segment size = 60 seconds](#toc1_7_1_2_)    
    - [Noisy ECG 2](#toc1_7_2_)    
      - [Segment size = 300 seconds](#toc1_7_2_1_)    
      - [Segment size = 60 seconds](#toc1_7_2_2_)    
    - [Unusable ECG](#toc1_7_3_)    
      - [Segment size = 300 seconds](#toc1_7_3_1_)    
      - [Segment size = 60 seconds](#toc1_7_3_2_)    
  - [**Compare ECG and PPG signals from the same session**](#toc1_8_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[**Import libraries**](#toc0_)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sys import path
path.append('./core/')
from heartview_sqa_cardio import Cardio
import ECG
import PPG
path.append('./utils/')
import data_visualization as dv
import signal_quality_analysis as sqa

## <a id='toc1_2_'></a>[**Setting parameters**](#toc0_)

In [2]:
# Data directory
data_dir = './data/'
mims_data_dir = './data/mims/'

# Data files
clean_mindware_file = 'clean_mindware_ecg_1000.csv'
clean_movisens_file = 'clean_movisens_ecg_1024.csv'
empatica_1_file = 'empatica-1_bvp_64.csv'
empatica_2_file = 'empatica-2_bvp_64.csv'
empatica_3_file = 'empatica-2_bvp_64.csv'
noisy_actiwave_1_file = 'noisy_actiwave_ecg_acc-1_1024.csv'
noisy_actiwave_2_file = 'noisy_actiwave_ecg_acc-2_1024.csv'
unusable_ecg_file = 'unusable_ecg_acc_1024.csv'

# MIMS data
mims_actiwave_1_file = 'noisy_actiwave_ecg_acc-1_1024_mims.csv'
mims_actiwave_2_file = 'noisy_actiwave_ecg_acc-2_1024_mims.csv'
mims_unusable_ecg_file = 'unusable_ecg_acc_1024_mims.csv'

In [3]:
# Set sampling frequencies
fs_mindware = 1000
fs_movisens = 1024
fs_empatica = 64
fs_actiwave = 1024
fs_unusable_ecg = 1024

## <a id='toc1_3_'></a>[**Visualize raw data**](#toc0_)

First, plot raw data and check signal quality.

In [4]:
def beat_detection_ecg(df: pd.DataFrame, fs: int, colname: str):
    # Filter ECG
    Filter = ECG.Filters(fs=fs, powerline_freq=60)
    df['Filtered ECG'] = Filter.filter_signal(df[colname])

    # Detect R-peaks
    BeatDetector = ECG.BeatDetectors(fs=fs, preprocessed=True)
    ecg_beats = BeatDetector.manikandan(signal=df['Filtered ECG'], adaptive_threshold=True)
    df['Peak'] = np.nan
    df.loc[df.index.isin(ecg_beats), 'Peak'] = 1

    return ecg_beats, df

def beat_detection_ppg(df: pd.DataFrame, fs: int, colname: str):
    # Filter PPG
    Filter = PPG.Filters(fs=fs)
    df['Filtered BVP'] = Filter.filter_signal(df[colname])

    # Detect peaks
    BeatDetector = PPG.BeatDetectors(fs=fs, preprocessed=True)
    ppg_beats = BeatDetector.erma(signal=df['Filtered BVP'])
    df['Peak'] = np.nan
    df.loc[df.index.isin(ppg_beats), 'Peak'] = 1
    
    return ppg_beats, df

### <a id='toc1_3_1_'></a>[Clean ECG](#toc0_)

In [5]:
# Load data
clean_mindware = pd.read_csv(data_dir + clean_mindware_file)

# Beat detection
ecg_beats_clean_mindware, clean_mindware = beat_detection_ecg(df=clean_mindware, fs=fs_mindware, colname='ECG')

clean_mindware

,Time,ECG,Filtered ECG,Peak
0,2024-10-09 12:34:00.000,0.005951,0.000702,NaN
1,2024-10-09 12:34:00.001,0.005646,0.000939,NaN
2,2024-10-09 12:34:00.002,0.008392,0.001171,NaN
3,2024-10-09 12:34:00.003,0.010681,0.001393,NaN
4,2024-10-09 12:34:00.004,0.008850,0.001601,NaN
...,...,...,...,...
959995,2024-10-09 12:49:59.995,0.016937,0.005341,NaN
959996,2024-10-09 12:49:59.996,0.014496,0.004752,NaN
959997,2024-10-09 12:49:59.997,0.013733,0.004184,NaN
959998,2024-10-09 12:49:59.998,0.015411,0.003640,NaN


In [6]:
# Plot raw data
dv.plot_raw_interactive(data = clean_mindware, colname = 'Filtered ECG', time_colname = 'Time', peak_colname='Peak', fs = fs_mindware, seg_sec = 60)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=16, min=1), Out…

### <a id='toc1_3_2_'></a>[Noisy PPG](#toc0_)

In [7]:
# Load data
empatica_1 = pd.read_csv(data_dir + empatica_1_file)

# Beat detection
ppg_beats_empatica_1, empatica_1 = beat_detection_ppg(df=empatica_1, fs=fs_empatica, colname='BVP')

empatica_1

,Time,BVP,Filtered BVP,Peak
0,2021-11-15 22:59:26.000000,99.67,-23.316591,NaN
1,2021-11-15 22:59:26.015625,99.74,-29.033751,NaN
2,2021-11-15 22:59:26.031250,99.43,-34.843487,NaN
3,2021-11-15 22:59:26.046875,97.86,-40.644369,NaN
4,2021-11-15 22:59:26.062500,93.65,-46.355076,NaN
...,...,...,...,...
76795,2021-11-15 23:19:25.921875,69.24,-34.630141,NaN
76796,2021-11-15 23:19:25.937500,77.50,-31.502971,NaN
76797,2021-11-15 23:19:25.953125,84.17,-27.989845,NaN
76798,2021-11-15 23:19:25.968750,89.12,-24.166435,NaN


In [8]:
# Plot raw data
dv.plot_raw_interactive(data = empatica_1, colname = 'Filtered BVP', time_colname = 'Time', peak_colname='Peak', fs = fs_empatica, seg_sec = 60)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=20, min=1), Out…

### <a id='toc1_3_3_'></a>[Noisy PPG 2](#toc0_)

In [9]:
# Load data
empatica_2 = pd.read_csv(data_dir + empatica_2_file)

# Beat detection
ppg_beats_empatica_2, empatica_2 = beat_detection_ppg(df=empatica_2, fs=fs_empatica, colname='BVP')

empatica_2

,Time,BVP,Filtered BVP,Peak
0,2021-11-16 19:46:04.000000,7.08,-9.445512,NaN
1,2021-11-16 19:46:04.015625,4.10,-10.536238,NaN
2,2021-11-16 19:46:04.031250,1.03,-11.603082,NaN
3,2021-11-16 19:46:04.046875,-1.95,-12.591670,NaN
4,2021-11-16 19:46:04.062500,-4.74,-13.450931,NaN
...,...,...,...,...
57595,2021-11-16 20:01:03.921875,21.14,11.403494,NaN
57596,2021-11-16 20:01:03.937500,21.08,11.128109,NaN
57597,2021-11-16 20:01:03.953125,20.54,10.736702,NaN
57598,2021-11-16 20:01:03.968750,19.33,10.248353,NaN


In [10]:
# Plot raw data
dv.plot_raw_interactive(data = empatica_2, colname = 'Filtered BVP', time_colname = 'Time', peak_colname='Peak', fs = fs_empatica, seg_sec = 60)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=15, min=1), Out…

### Noisy PPG 3

In [11]:
# Load data
empatica_3 = pd.read_csv(data_dir + empatica_3_file)

# Beat detection
ppg_beats_empatica_3, empatica_3 = beat_detection_ppg(df=empatica_3, fs=fs_empatica, colname='BVP')

empatica_3

,Time,BVP,Filtered BVP,Peak
0,2021-11-16 19:46:04.000000,7.08,-9.445512,NaN
1,2021-11-16 19:46:04.015625,4.10,-10.536238,NaN
2,2021-11-16 19:46:04.031250,1.03,-11.603082,NaN
3,2021-11-16 19:46:04.046875,-1.95,-12.591670,NaN
4,2021-11-16 19:46:04.062500,-4.74,-13.450931,NaN
...,...,...,...,...
57595,2021-11-16 20:01:03.921875,21.14,11.403494,NaN
57596,2021-11-16 20:01:03.937500,21.08,11.128109,NaN
57597,2021-11-16 20:01:03.953125,20.54,10.736702,NaN
57598,2021-11-16 20:01:03.968750,19.33,10.248353,NaN


In [12]:
# Plot raw data
dv.plot_raw_interactive(data = empatica_3, colname = 'Filtered BVP', time_colname = 'Time', peak_colname='Peak', fs = fs_empatica, seg_sec = 60)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=15, min=1), Out…

### <a id='toc1_3_4_'></a>[Noisy ECG](#toc0_)

In [13]:
# Load data
noisy_actiwave_1 = pd.read_csv(data_dir + noisy_actiwave_1_file)

# Beat detection
ecg_beats_actiwave_1, noisy_actiwave_1 = beat_detection_ecg(df=noisy_actiwave_1, fs=fs_actiwave, colname='ECG')

noisy_actiwave_1

,Timestamp,ECG,X,Y,Z,Filtered ECG,Peak
0,2017-03-18 14:57:18.000000,0.110697,4.167150,7.967184,-3.331023,0.008806,NaN
1,2017-03-18 14:57:18.000977,0.110697,4.156314,7.967049,-3.327827,0.004319,NaN
2,2017-03-18 14:57:18.001953,0.102508,4.145660,7.967226,-3.324982,-0.000172,NaN
3,2017-03-18 14:57:18.002930,0.078527,4.135242,7.967687,-3.322463,-0.004629,NaN
4,2017-03-18 14:57:18.003906,0.081159,4.125112,7.968403,-3.320242,-0.009015,NaN
...,...,...,...,...,...,...,...
2004988,2017-03-18 15:29:55.996094,-0.063608,8.005245,5.841280,-1.693565,-0.022305,NaN
2004989,2017-03-18 15:29:55.997070,-0.079401,7.997391,5.832431,-1.717957,-0.020363,NaN
2004990,2017-03-18 15:29:55.998047,-0.066533,7.988913,5.823656,-1.741808,-0.018277,NaN
2004991,2017-03-18 15:29:55.999023,-0.035825,7.979833,5.814984,-1.765055,-0.016088,NaN


In [14]:
# Plot raw data
dv.plot_raw_interactive(data = noisy_actiwave_1, colname = 'Filtered ECG', time_colname = 'Timestamp', peak_colname='Peak', fs = fs_actiwave, seg_sec = 60)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=33, min=1), Out…

### <a id='toc1_3_5_'></a>[Noisy ECG 2](#toc0_)

In [15]:
# Load data
noisy_actiwave_2 = pd.read_csv(data_dir + noisy_actiwave_2_file)

# Beat detection
ecg_beats_actiwave_2, noisy_actiwave_2 = beat_detection_ecg(df=noisy_actiwave_2, fs=fs_actiwave, colname='ECG')

noisy_actiwave_2

,Timestamp,ECG,X,Y,Z,Filtered ECG,Peak
0,2017-01-16 11:34:31.000000,0.049573,-3.253104,9.696379,0.259229,0.020803,NaN
1,2017-01-16 11:34:31.000977,0.050743,-3.236605,9.683939,0.260479,0.020088,NaN
2,2017-01-16 11:34:31.001953,0.027054,-3.219710,9.671238,0.261524,0.019438,NaN
3,2017-01-16 11:34:31.002930,0.021497,-3.202473,9.658296,0.262367,0.018868,NaN
4,2017-01-16 11:34:31.003906,0.040215,-3.184956,9.645136,0.263012,0.018392,NaN
...,...,...,...,...,...,...,...
1035260,2017-01-16 11:51:21.996094,-0.009796,-5.255432,7.226353,2.816282,0.019333,NaN
1035261,2017-01-16 11:51:21.997070,-0.008626,-5.244926,7.209909,2.838964,0.016945,NaN
1035262,2017-01-16 11:51:21.998047,-0.008041,-5.234835,7.192982,2.861934,0.014738,NaN
1035263,2017-01-16 11:51:21.999023,0.000440,-5.225199,7.175660,2.885124,0.012710,NaN


In [16]:
dv.plot_raw_interactive(data = noisy_actiwave_2, colname = 'Filtered ECG', time_colname = 'Timestamp', peak_colname='Peak', fs = fs_actiwave, seg_sec = 60)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=17, min=1), Out…

### <a id='toc1_3_6_'></a>[Unusable ECG](#toc0_)

In [17]:
# Load data
unusable_ecg = pd.read_csv(data_dir + unusable_ecg_file)

# Filter ECG
Filter = ECG.Filters(fs=fs_unusable_ecg, powerline_freq=60)
unusable_ecg['Filtered ECG'] = Filter.filter_signal(unusable_ecg['ECG'])

# Detect R-peaks
BeatDetector = ECG.BeatDetectors(fs=fs_unusable_ecg, preprocessed=True)
ecg_beats_unusable_ecg = BeatDetector.manikandan(signal=unusable_ecg['Filtered ECG'], adaptive_threshold=True)
unusable_ecg['Peak'] = np.nan
unusable_ecg.loc[unusable_ecg.index.isin(ecg_beats_unusable_ecg), 'Peak'] = 1

unusable_ecg

,Timestamp,ECG,X,Y,Z,Filtered ECG,Peak
0,2016-09-21 18:04:00.000000,0.089055,-3.235123,4.676619,-8.054093,0.058416,NaN
1,2016-09-21 18:04:00.000977,0.096367,-3.229053,4.684367,-8.055504,0.059369,NaN
2,2016-09-21 18:04:00.001953,0.103678,-3.222176,4.692811,-8.056672,0.060286,NaN
3,2016-09-21 18:04:00.002930,0.087008,-3.214526,4.701869,-8.057613,0.061156,NaN
4,2016-09-21 18:04:00.003906,0.097244,-3.206142,4.711456,-8.058348,0.061971,NaN
...,...,...,...,...,...,...,...
921595,2016-09-21 18:18:59.995117,5.009083,13.810299,1.153427,1.392078,0.825561,NaN
921596,2016-09-21 18:18:59.996094,4.803485,13.934142,1.157580,1.450984,0.789405,NaN
921597,2016-09-21 18:18:59.997070,4.663105,14.039550,1.162269,1.509638,0.741776,NaN
921598,2016-09-21 18:18:59.998047,4.600226,14.126216,1.167466,1.567611,0.684889,NaN


In [18]:
# Plot raw data
dv.plot_raw_interactive(data = unusable_ecg, colname = 'Filtered ECG', time_colname = 'Timestamp', peak_colname='Peak', fs = fs_unusable_ecg, seg_sec = 60)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=15, min=1), Out…

## <a id='toc1_4_'></a>[**Visualize artifacts**](#toc0_)

Here, we detect artifactual beats and visualize their locations.

### <a id='toc1_4_1_'></a>[**Artifact detection methods**](#toc0_)

Two types of artifact detection methods are available.

1. **Hegarty-Craver et al.'s algorithm**

    You can use this method by setting the method parameter as `method='hegarty'` in the `cardio.identify_artifacts()` function.

    This method classifies each IBI as short/correct/long/extra long by comparing each IBI against the estimated IBI (i.e., median of the previous IBIs).

2. **Quigley et al.'s algorithm**

    You can use this method by setting the method parameter as `method='cbd'` in the `cardio.identify_artifacts()` function.

    This method estimates the largest expected beat difference among normal beats and the smallest expected beat difference among artifactual beats based on the distribution. It then identifies artifactual beats based on the calculated criterion. 

3. **Both algorithms**

    You can use both algorithms at the same time by setting the method parameter as `method='both'` in the `cardio.identify_artifacts()` function.

    If this option is selected, beats identified as artifacts by either detection method are returned as artifacts.

Reference: 
> Hegarty-Craver, M., Gilchrist, K. H., Propper, C. B., Lewis, G. F., DeFilipp, S. J., Coffman, J. L., & Willoughby, M. T. (2018). Automated respiratory sinus arrhythmia measurement: Demonstration using executive function assessment. Behavior Research Methods, 50(5), 1816–1823.
  
> Berntson, G., Quigley, K., Jang, J., Boysen, S. (1990). An approach to
artifact identification: Application to heart period data.
Psychophysiology, 27(5), 586–598.

### <a id='toc1_4_2_'></a>[Clean ECG](#toc0_)

In [19]:
cardio = Cardio(fs=fs_mindware)
artifacts_clean_mindware = cardio.identify_artifacts(beats_ix=ecg_beats_clean_mindware, method='hegarty')
dv.plot_artifacts_interactive(data=clean_mindware, colname='Filtered ECG', time_colname='Time', artifacts_idx=artifacts_clean_mindware, fs = fs_mindware, seg_sec = 60)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=16, min=1), Out…

### <a id='toc1_4_3_'></a>[Noisy PPG](#toc0_)

In [20]:
cardio = Cardio(fs=fs_empatica)
artifacts_empatica_1 = cardio.identify_artifacts(beats_ix=ppg_beats_empatica_1, method='hegarty')
dv.plot_artifacts_interactive(data=empatica_1, colname='Filtered BVP', time_colname='Time', artifacts_idx=artifacts_empatica_1, fs = fs_empatica, seg_sec = 60)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=20, min=1), Out…

### <a id='toc1_4_4_'></a>[Noisy PPG 2](#toc0_)

In [21]:
cardio = Cardio(fs=fs_empatica)
artifacts_empatica_2 = cardio.identify_artifacts(beats_ix=ppg_beats_empatica_2, method='hegarty')
dv.plot_artifacts_interactive(data=empatica_2, colname='Filtered BVP', time_colname='Time', artifacts_idx=artifacts_empatica_2, fs = fs_empatica, seg_sec = 60)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=15, min=1), Out…

### Noisy PPG 3

In [22]:
cardio = Cardio(fs=fs_empatica)
artifacts_empatica_3 = cardio.identify_artifacts(beats_ix=ppg_beats_empatica_3, method='hegarty')
dv.plot_artifacts_interactive(data=empatica_3, colname='Filtered BVP', time_colname='Time', artifacts_idx=artifacts_empatica_3, fs = fs_empatica, seg_sec = 60)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=15, min=1), Out…

### <a id='toc1_4_5_'></a>[Noisy ECG](#toc0_)

In [23]:
cardio = Cardio(fs=fs_actiwave)
artifacts_actiwave_1 = cardio.identify_artifacts(beats_ix=ecg_beats_actiwave_1, method='hegarty')
dv.plot_artifacts_interactive(data=noisy_actiwave_1, colname='Filtered ECG', time_colname='Timestamp', artifacts_idx=artifacts_actiwave_1, fs = fs_actiwave, seg_sec = 60)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=33, min=1), Out…

### <a id='toc1_4_6_'></a>[Noisy ECG 2](#toc0_)

In [24]:
cardio = Cardio(fs=fs_actiwave)
artifacts_actiwave_2 = cardio.identify_artifacts(beats_ix=ecg_beats_actiwave_2, method='hegarty')
dv.plot_artifacts_interactive(data=noisy_actiwave_2, colname='Filtered ECG', time_colname='Timestamp', artifacts_idx=artifacts_actiwave_2, fs = fs_actiwave, seg_sec = 60)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=17, min=1), Out…

### <a id='toc1_4_7_'></a>[Unusable ECG](#toc0_)

In [25]:
cardio = Cardio(fs=fs_unusable_ecg)
artifacts_unusable_ecg = cardio.identify_artifacts(beats_ix=ecg_beats_unusable_ecg, method='hegarty')
dv.plot_artifacts_interactive(data=unusable_ecg, colname='Filtered ECG', time_colname='Timestamp', artifacts_idx=artifacts_unusable_ecg, fs = fs_unusable_ecg, seg_sec = 60)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=15, min=1), Out…

## <a id='toc1_5_'></a>[**Beat correction**](#toc0_)

Then, correct artifactual beats with an algorithm.

The algorithm used here is developed by Hegarty-Craver et al. (2018).

Using the same criteria in the artifact detection algorithm, this method classifies each IBI as short/correct/long/extra long. It checks the flags of the previous and current IBIs and applies different correction methods based on their combination.

**Correction methods used**
1. **Accept**—Accept without correction.
2. **Add**—Add previous and current IBIs.
3. **Average**—Average previous and current IBIs.
4. **Split**—Add previous and current IBIs and split the values into more than three.

Reference: 
> Hegarty-Craver, M., Gilchrist, K. H., Propper, C. B., Lewis, G. F., DeFilipp, S. J., Coffman, J. L., & Willoughby, M. T. (2018). Automated respiratory sinus arrhythmia measurement: Demonstration using executive function assessment. Behavior Research Methods, 50(5), 1816–1823.
  



In [26]:
def beat_correction(df: pd.DataFrame, fs: int, beats_ix: list, seg_size: int):
    cardio = Cardio(fs=fs)
    original, corrected = cardio.correct_interval(beats_ix=beats_ix, seg_size=seg_size)
    df['Peak after correction'] = np.nan
    df.loc[df.index.isin(corrected['Corrected Beat']), 'Peak after correction'] = 1

    return corrected, df

### <a id='toc1_5_1_'></a>[Clean ECG](#toc0_)

In [27]:
corrected_clean_mindware, clean_mindware = beat_correction(df=clean_mindware, fs=fs_mindware, beats_ix=ecg_beats_clean_mindware, seg_size=60)
dv.plot_correction_interactive(data=clean_mindware, colname='Filtered ECG', time_colname='Time', peak_colname_correction='Peak after correction', fs=fs_mindware, seg_sec=60)

Estimated average HR (bpm):  91


interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=16, min=1), Out…

### <a id='toc1_5_2_'></a>[Noisy PPG](#toc0_)

In [28]:
corrected_empatica_1, empatica_1 = beat_correction(df=empatica_1, fs=fs_empatica, beats_ix=ppg_beats_empatica_1, seg_size=60)
dv.plot_correction_interactive(data=empatica_1, colname='Filtered BVP', time_colname='Time', peak_colname_correction='Peak after correction', fs=fs_empatica, seg_sec=60)

Estimated average HR (bpm):  70


/Users/yuna.w/Research/CBSL/heartview_workshop/.venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/yuna.w/Research/CBSL/heartview_workshop/.venv/lib/python3.12/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=20, min=1), Out…

### <a id='toc1_5_3_'></a>[Noisy PPG 2](#toc0_)

In [29]:
corrected_empatica_2, empatica_2 = beat_correction(df=empatica_2, fs=fs_empatica, beats_ix=ppg_beats_empatica_2, seg_size=60)
dv.plot_correction_interactive(data=empatica_2, colname='Filtered BVP', time_colname='Time', peak_colname_correction='Peak after correction', fs=fs_empatica, seg_sec=60)

Estimated average HR (bpm):  67


interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=15, min=1), Out…

### Noisy PPG 3

In [30]:
corrected_empatica_3, empatica_3 = beat_correction(df=empatica_3, fs=fs_empatica, beats_ix=ppg_beats_empatica_3, seg_size=60)
dv.plot_correction_interactive(data=empatica_3, colname='Filtered BVP', time_colname='Time', peak_colname_correction='Peak after correction', fs=fs_empatica, seg_sec=60)

Estimated average HR (bpm):  67


interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=15, min=1), Out…

### <a id='toc1_5_4_'></a>[Noisy ECG](#toc0_)

In [31]:
corrected_actiwave_1, noisy_actiwave_1 = beat_correction(df=noisy_actiwave_1, fs=fs_actiwave, beats_ix=ecg_beats_actiwave_1, seg_size=60)
dv.plot_correction_interactive(data=noisy_actiwave_1, colname='Filtered ECG', time_colname='Timestamp', peak_colname_correction='Peak after correction', fs=fs_actiwave, seg_sec=60)

Estimated average HR (bpm):  140


interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=33, min=1), Out…

### <a id='toc1_5_5_'></a>[Noisy ECG 2](#toc0_)

In [32]:
corrected_actiwave_2, noisy_actiwave_2 = beat_correction(df=noisy_actiwave_2, fs=fs_actiwave, beats_ix=ecg_beats_actiwave_2, seg_size=60)
dv.plot_correction_interactive(data=noisy_actiwave_2, colname='Filtered ECG', time_colname='Timestamp', peak_colname_correction='Peak after correction', fs=fs_actiwave, seg_sec=60)

Estimated average HR (bpm):  90


interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=17, min=1), Out…

### <a id='toc1_5_6_'></a>[Unusable ECG](#toc0_)

In [33]:
corrected_unusable_ecg, unusable_ecg = beat_correction(df=unusable_ecg, fs=fs_unusable_ecg, beats_ix=ecg_beats_unusable_ecg, seg_size=60)
dv.plot_correction_interactive(data=unusable_ecg, colname='Filtered ECG', time_colname='Timestamp', peak_colname_correction='Peak after correction', fs=fs_unusable_ecg, seg_sec=60)

Estimated average HR (bpm):  121


interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=15, min=1), Out…

## <a id='toc1_6_'></a>[**Visualize the association between acceleration data and signal quality**](#toc0_)

Next, visualize the association between acceleration data (MIMS score) collected from devices and the signal quality to see how motion artifacts can impact signal quality, depending on datasets.

> MIMS-unit is abbreviated for Monitor Independent Movement Summary unit. This measurement is developed to harmonize the processing of accelerometer data from different devices.

MIMS scores were calculated using [MIMSunit package](https://mhealthgroup.github.io/MIMSunit/index.html) developed by mHealth Research Group at Northeastern University.

### <a id='toc1_6_1_'></a>[Noisy ECG](#toc0_)

#### <a id='toc1_6_1_1_'></a>[Segment size = 300 seconds](#toc0_)

First, take a look at a larger segment size to get a big picture.

In [34]:
mims_actiwave_1 = pd.read_csv(mims_data_dir + mims_actiwave_1_file)
mims_actiwave_1['HEADER_TIME_STAMP'] = pd.to_datetime(mims_actiwave_1['HEADER_TIME_STAMP'])
dv.plot_mims_artifacts_interactive(data=noisy_actiwave_1, mims = mims_actiwave_1, colname='Filtered ECG', time_colname='Timestamp', artifacts_idx=artifacts_actiwave_1, peak_colname='Peak', fs=fs_actiwave, seg_sec=300, mims_threshold=0.1)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=7, min=1), Outp…

#### <a id='toc1_6_1_2_'></a>[Segment size = 60 seconds](#toc0_)

Zoom in to inspect more closely.

In [35]:
mims_actiwave_1 = pd.read_csv(mims_data_dir + mims_actiwave_1_file)
mims_actiwave_1['HEADER_TIME_STAMP'] = pd.to_datetime(mims_actiwave_1['HEADER_TIME_STAMP'])
dv.plot_mims_artifacts_interactive(data=noisy_actiwave_1, mims = mims_actiwave_1, colname='Filtered ECG', time_colname='Timestamp', artifacts_idx=artifacts_actiwave_1, peak_colname='Peak', fs=fs_actiwave, seg_sec=60, mims_threshold=0.1)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=33, min=1), Out…

### <a id='toc1_6_2_'></a>[Noisy ECG 2](#toc0_)

#### <a id='toc1_6_2_1_'></a>[Segment size = 300 seconds](#toc0_)

In [36]:
mims_actiwave_2 = pd.read_csv(mims_data_dir + mims_actiwave_2_file)
mims_actiwave_2['HEADER_TIME_STAMP'] = pd.to_datetime(mims_actiwave_2['HEADER_TIME_STAMP'])
dv.plot_mims_artifacts_interactive(data=noisy_actiwave_2, mims = mims_actiwave_2, colname='Filtered ECG', time_colname='Timestamp', artifacts_idx=artifacts_actiwave_2, peak_colname='Peak', fs=fs_actiwave, seg_sec=300, mims_threshold=0.2)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=4, min=1), Outp…

#### <a id='toc1_6_2_2_'></a>[Segment size = 60 seconds](#toc0_)

Zoom in to inspect more closely.

In [37]:
mims_actiwave_2 = pd.read_csv(mims_data_dir + mims_actiwave_2_file)
mims_actiwave_2['HEADER_TIME_STAMP'] = pd.to_datetime(mims_actiwave_2['HEADER_TIME_STAMP'])
dv.plot_mims_artifacts_interactive(data=noisy_actiwave_2, mims = mims_actiwave_2, colname='Filtered ECG', time_colname='Timestamp', artifacts_idx=artifacts_actiwave_2, peak_colname='Peak', fs=fs_actiwave, seg_sec=60, mims_threshold=0.2)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=17, min=1), Out…

### <a id='toc1_6_3_'></a>[Unusable ECG](#toc0_)

#### <a id='toc1_6_3_1_'></a>[Segment size = 300 seconds](#toc0_)

In [38]:
mims_unusable_ecg = pd.read_csv(mims_data_dir + mims_unusable_ecg_file)
mims_unusable_ecg['HEADER_TIME_STAMP'] = pd.to_datetime(mims_unusable_ecg['HEADER_TIME_STAMP'])
dv.plot_mims_artifacts_interactive(data=unusable_ecg, mims = mims_unusable_ecg, colname='Filtered ECG', time_colname='Timestamp', artifacts_idx=artifacts_unusable_ecg, peak_colname='Peak', fs=fs_unusable_ecg, seg_sec=300, mims_threshold=0.2)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=3, min=1), Outp…

#### <a id='toc1_6_3_2_'></a>[Segment size = 60 seconds](#toc0_)

Zoom in to inspect more closely.

In [39]:
mims_unusable_ecg = pd.read_csv(mims_data_dir + mims_unusable_ecg_file)
mims_unusable_ecg['HEADER_TIME_STAMP'] = pd.to_datetime(mims_unusable_ecg['HEADER_TIME_STAMP'])
dv.plot_mims_artifacts_interactive(data=unusable_ecg, mims = mims_unusable_ecg, colname='Filtered ECG', time_colname='Timestamp', artifacts_idx=artifacts_unusable_ecg, peak_colname='Peak', fs=fs_unusable_ecg, seg_sec=60, mims_threshold=0.2)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=15, min=1), Out…

## <a id='toc1_7_'></a>[**Visualize the association between acceleration data and beat correction**](#toc0_)

### <a id='toc1_7_1_'></a>[Noisy ECG](#toc0_)

#### <a id='toc1_7_1_1_'></a>[Segment size = 300 seconds](#toc0_)

First, take a look at a larger segment size to get a big picture.

In [40]:
dv.plot_mims_correction_interactive(data=noisy_actiwave_1, mims = mims_actiwave_1, colname='Filtered ECG', time_colname='Timestamp', peak_colname='Peak', peak_colname_correction='Peak after correction', fs=fs_actiwave, seg_sec=300, mims_threshold=0.2)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=7, min=1), Outp…

#### <a id='toc1_7_1_2_'></a>[Segment size = 60 seconds](#toc0_)

Zoom in to inspect more closely.

In [41]:
dv.plot_mims_correction_interactive(data=noisy_actiwave_1, mims = mims_actiwave_1, colname='Filtered ECG', time_colname='Timestamp', peak_colname='Peak', peak_colname_correction='Peak after correction', fs=fs_actiwave, seg_sec=60, mims_threshold=0.2)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=33, min=1), Out…

### <a id='toc1_7_2_'></a>[Noisy ECG 2](#toc0_)

#### <a id='toc1_7_2_1_'></a>[Segment size = 300 seconds](#toc0_)

First, take a look at a larger segment size to get a big picture.

In [42]:
dv.plot_mims_correction_interactive(data=noisy_actiwave_2, mims = mims_actiwave_2, colname='Filtered ECG', time_colname='Timestamp', peak_colname='Peak', peak_colname_correction='Peak after correction', fs=fs_actiwave, seg_sec=300, mims_threshold=0.2)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=4, min=1), Outp…

#### <a id='toc1_7_2_2_'></a>[Segment size = 60 seconds](#toc0_)

Zoom in to inspect more closely.

In [43]:
dv.plot_mims_correction_interactive(data=noisy_actiwave_2, mims = mims_actiwave_2, colname='Filtered ECG', time_colname='Timestamp', peak_colname='Peak', peak_colname_correction='Peak after correction', fs=fs_actiwave, seg_sec=60, mims_threshold=0.2)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=17, min=1), Out…

### <a id='toc1_7_3_'></a>[Unusable ECG](#toc0_)

#### <a id='toc1_7_3_1_'></a>[Segment size = 300 seconds](#toc0_)

First, take a look at a larger segment size to get a big picture.

In [44]:
dv.plot_mims_correction_interactive(data=unusable_ecg, mims = mims_unusable_ecg, colname='Filtered ECG', time_colname='Timestamp', peak_colname='Peak', peak_colname_correction='Peak after correction', fs=fs_unusable_ecg, seg_sec=300, mims_threshold=0.2)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=3, min=1), Outp…

#### <a id='toc1_7_3_2_'></a>[Segment size = 60 seconds](#toc0_)

Zoom in to inspect more closely.

In [45]:
dv.plot_mims_correction_interactive(data=unusable_ecg, mims = mims_unusable_ecg, colname='Filtered ECG', time_colname='Timestamp', peak_colname='Peak', peak_colname_correction='Peak after correction', fs=fs_unusable_ecg, seg_sec=60, mims_threshold=0.2)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=15, min=1), Out…

## <a id='toc1_8_'></a>[**Compare ECG and PPG signals from the same session**](#toc0_)

In the following example, we use PPG and ECG data for the same participant and session in the WESAD dataset. 

Here, ECG signals are used as ground truth to evaluate the quality of PPG signals and correction performance. We followed Charlton et al.'s method to compare beat locations between ECG and PPG signals and marked beats as "agreement" when PPG beat locations fell within 150 ms from ECG beat locations.

Reference:
> Schmidt, P., Reiss, A., Duerichen, R., Marberger, C., & Van Laerhoven, K. (2018). Introducing WESAD, a multimodal dataset for wearable stress and affect detection. Proceedings of the 20th ACM International Conference on Multimodal Interaction, 400–408.

> Charlton, P. H., Kotzen, K., Mejía-Mejía, E., Aston, P. J., Budidha, K., Mant, J., Pettit, C., Behar, J. A., & Kyriacou, P. A. (2022). Detecting beats in the photoplethysmogram: benchmarking open-source algorithms. Physiological Measurement, 43(8). https://doi.org/10.1088/1361-6579/ac826d



In [46]:
# Read PPG data
fs_wesad_ppg = 64
wesad_ppg = pd.read_csv('./data/wesad_ppg.csv')
wesad_ppg

,BVP,ACC_x,ACC_y,ACC_z,Timestamp,Label,Filtered,Peak
0,4.72,NaN,NaN,NaN,2017-07-25 14:41:31.859375,3,5.347591,NaN
1,4.58,-6.0,-7.0,63.0,2017-07-25 14:41:31.875000,3,6.180398,NaN
2,4.43,NaN,NaN,NaN,2017-07-25 14:41:31.890625,3,6.774856,NaN
3,4.40,-6.0,-7.0,63.0,2017-07-25 14:41:31.906250,3,7.142300,NaN
4,4.53,NaN,NaN,NaN,2017-07-25 14:41:31.921875,3,7.315290,NaN
...,...,...,...,...,...,...,...,...
23554,-8.48,NaN,NaN,NaN,2017-07-25 14:47:39.890625,3,-11.881672,NaN
23555,-3.94,3.0,-57.0,26.0,2017-07-25 14:47:39.906250,3,-9.927519,NaN
23556,0.77,NaN,NaN,NaN,2017-07-25 14:47:39.921875,3,-7.788052,NaN
23557,5.32,3.0,-57.0,26.0,2017-07-25 14:47:39.937500,3,-5.514005,NaN


In [47]:
# Read ECG data
fs_wesad_ecg = 700
wesad_ecg = pd.read_csv('./data/wesad_ecg.csv')
wesad_ecg

,Timestamp,ECG,Filtered,Peak,Label
0,2017-07-25 14:41:31.857173,-0.066788,-0.061821,NaN,3
1,2017-07-25 14:41:31.858602,-0.064133,-0.061818,NaN,3
2,2017-07-25 14:41:31.860031,-0.060287,-0.061758,NaN,3
3,2017-07-25 14:41:31.861460,-0.060379,-0.061647,NaN,3
4,2017-07-25 14:41:31.862889,-0.062943,-0.061489,NaN,3
...,...,...,...,...,...
257595,2017-07-25 14:47:39.960428,-0.090500,-0.097039,NaN,3
257596,2017-07-25 14:47:39.961857,-0.092789,-0.100334,NaN,3
257597,2017-07-25 14:47:39.963286,-0.095947,-0.103266,NaN,3
257598,2017-07-25 14:47:39.964715,-0.099655,-0.105843,NaN,3


In [48]:
# Read MIMS data
wesad_mims = pd.read_csv('./data/MIMS/wesad_mims.csv')
wesad_mims

,Timestamp,MIMS_UNIT,MIMS_UNIT_X,MIMS_UNIT_Y,MIMS_UNIT_Z
0,2017-07-25 14:41:32.000500,0.000000,0.000000,0.0,0.000000
1,2017-07-25 14:41:33.000500,0.000000,0.000000,0.0,0.000000
2,2017-07-25 14:41:34.000500,0.000000,0.000000,0.0,0.000000
3,2017-07-25 14:41:35.000500,0.000000,0.000000,0.0,0.000000
4,2017-07-25 14:41:36.000500,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...
363,2017-07-25 14:47:35.000500,0.027309,0.014661,0.0,0.012648
364,2017-07-25 14:47:36.000500,0.010726,0.010726,0.0,0.000000
365,2017-07-25 14:47:37.000500,0.012398,0.012398,0.0,0.000000
366,2017-07-25 14:47:38.000500,0.000000,0.000000,0.0,0.000000


In [49]:
# Correct beats
wesad_beats_ix = wesad_ppg.loc[wesad_ppg['Peak'] == 1].index
corrected_wesad_ppg, wesad_ppg = beat_correction(df=wesad_ppg, fs=fs_wesad_ppg, beats_ix=wesad_beats_ix, seg_size=60)

Estimated average HR (bpm):  72


In [50]:
# Convert Timestamp to datetime
wesad_ppg['Timestamp'] = pd.to_datetime(wesad_ppg['Timestamp'])
wesad_ecg['Timestamp'] = pd.to_datetime(wesad_ecg['Timestamp'])
# Compare beat locations between ECG and PPG
result_df, df_ppg_combined, df_ecg_combined = sqa.compare_beat_locations_Charlton(df_ppg=wesad_ppg, df_ecg=wesad_ecg, lag_type='best')

In [51]:
# Plot ECG/PPG beats and beat location agreement along with accelerometer data
wesad_mims = wesad_mims.rename(columns={'Timestamp': 'HEADER_TIME_STAMP'})
wesad_mims['HEADER_TIME_STAMP'] = pd.to_datetime(wesad_mims['HEADER_TIME_STAMP'])
dv.plot_correct(df_ppg=df_ppg_combined, df_ecg=df_ecg_combined, fs_ppg=fs_wesad_ppg, fs_ecg=fs_wesad_ecg, plot_acc=True, plot_mims=True, mims=wesad_mims, mims_threshold=0.2, seg_size=60)

interactive(children=(IntSlider(value=1, continuous_update=False, description='start_seg', max=7, min=1), Outp…